In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv

In [2]:
# Load cleaned listings csv's for Oct 2022 and 2023
l_df23 = pd.read_csv("../listings_oct_2023_clean.csv")
l_df22 = pd.read_csv("../listings_oct_2022_clean.csv")

In [3]:
l_df23.columns

Index(['Unnamed: 0', 'id', 'host_id', 'host_listings_count',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'amenities',
       'minimum_nights', 'minimum_minimum_nights', 'review_scores_rating',
       'review_scores_location', 'license', 'reviews_per_month'],
      dtype='object')

In [4]:
l_df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40545 entries, 0 to 40544
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    40545 non-null  int64  
 1   id                            40545 non-null  int64  
 2   host_id                       40545 non-null  int64  
 3   host_listings_count           40522 non-null  float64
 4   neighbourhood_cleansed        40545 non-null  object 
 5   neighbourhood_group_cleansed  40545 non-null  object 
 6   latitude                      40545 non-null  float64
 7   longitude                     40545 non-null  float64
 8   property_type                 40545 non-null  object 
 9   room_type                     40545 non-null  object 
 10  accommodates                  40545 non-null  int64  
 11  amenities                     40545 non-null  object 
 12  minimum_nights                40545 non-null  int64  
 13  m

In [5]:
# Build Dataframe to analyze minimum night listings
# Create bins
bins = ["0","1-29","30","30+"]
dfs = [l_df22, l_df23]
year = ["2022", "2023"]
min_nights_df = pd.DataFrame()

min_nights_df["Minimum Nights"] = bins 
min_nights_df["2022"] = ""
min_nights_df["2023"] = ""
        
for i in range(len(dfs)):
    min_nights_df[year[i]][0] = dfs[i]["minimum_nights"][dfs[i]["minimum_nights"]==0].count()
    min_nights_df[year[i]][1] = dfs[i]["minimum_nights"][(dfs[i]["minimum_nights"]>=1) & (dfs[i]["minimum_nights"]<30)].count()
    min_nights_df[year[i]][2] = dfs[i]["minimum_nights"][dfs[i]["minimum_nights"]==30].count()
    min_nights_df[year[i]][3] = dfs[i]["minimum_nights"][dfs[i]["minimum_nights"]>30].count()
    
min_nights_df.head()

,Minimum Nights,2022,2023
0,0,0,0
1,1-29,21314,3227
2,30,17256,33273
3,30+,1975,2292
